### Change wav file path in wav.scp

In [52]:

uid2path = []
subset_name = 'test_radio_wfsnr1' #test_radio_chn2 test_radchn2_dist1
wav_scp = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/wav.scp'%(subset_name)
new_wavstr = '/wav_wb_snr1/' #'/wav_radchn2_dist1/'

with open(wav_scp, 'r') as f:
    for l in f.readlines():
        uid, upath = l.split()
        upath = upath.replace('/wav/', new_wavstr)
        uid2path.append(' '.join([uid, upath]))

In [53]:
uid2path[0]

'id10270-5r0dWxy17C8-00001-wfsnr1 /home/yangwenhao/dataset/voxceleb1/voxceleb1_wav/vox1_test_wav/wav_wb_snr1/id10270/5r0dWxy17C8/00001.wav'

In [54]:
with open(wav_scp, 'w') as f:
    for l in uid2path:
        f.write(l + '\n')

### Trials_suffix

In [55]:
trials = []
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/trials'%(subset_name)
suffix = '-wfsnr1' #'-rd2dist1' #-radchn2

with open(trials_f, 'r') as f:
    for l in f.readlines():
        a, b,c = l.split()
        a += suffix
        b += suffix
        
        trials.append(' '.join([a, b, c]))
        
trials_name = 'trials_wfsnr1'
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/%s'%(subset_name, trials_name)
with open(trials_f, 'w') as f:
    for l in trials:
        f.write(l + '\n')

### Trials_xcross

In [46]:
trials = set([])
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/test/trials'
trials_name = 'trials_wfsnr1cross'#'trials_rdist1cross' # 'trials_bandcross'

subset = 'test_orgchn2fsnr5'#'test_orgchn2dist1'
suffix = '-wfsnr1' #'-rd2dist1' #-radchn2

with open(trials_f, 'r') as f:
    for l in f.readlines():
        a, b, c = l.split()
        trials.add(' '.join([a+suffix, b, c]))   
        trials.add(' '.join([a, b+suffix, c]))
          
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/%s'%(subset, trials_name)
with open(trials_f, 'w') as f:
    for l in trials:
        f.write(l + '\n')

### Trials_xy

In [9]:
trials = set([])
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/test/trials'
trials_name = 'trials_radiodist1' # 'trials_bandcross'
subset = 'test_orgchn2dist1'

suffix1 = '-rd2dist1' #-radchn2
suffix2 = '-radchn2' #-radchn2

with open(trials_f, 'r') as f:
    for l in f.readlines():
        a, b,c = l.split()
        trials.add(' '.join([a+suffix1, b+suffix2, c]))   
        trials.add(' '.join([a+suffix2, b+suffix1, c]))
          
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/%s'%(subset, trials_name)
with open(trials_f, 'w') as f:
    for l in trials:
        f.write(l + '\n')

In [5]:
trials[0]

'id10270-x6uYqmx31kE-00001-rd2dist1 id10270-8jEAjG6SegY-00008-rd2dist1 target'

In [6]:
with open(trials_f, 'w') as f:
    for l in trials:
        f.write(l + '\n')

In [16]:
import soundfile as sf
import os
from tqdm import tqdm

In [14]:
wav,sr = sf.read('/home/yangwenhao/dataset/voxceleb1/voxceleb1_wav/vox1_test_wav/wav_radchn2/id10291/3q0vhiAP7i8/00001.wav')

In [15]:
errors = set([])
for l in uid2path:
    uid, upath = l.split()
    try:
        wav,sr = sf.read(upath)
    except Exception as e:
        errors.add(upath)

In [17]:
errors

set()

In [19]:
trials_uids = set([])
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/test/trials'

with open(trials_f, 'r') as f:
    for l in f.readlines():
        a, b, c = l.split()
        trials_uids.add(a)
        trials_uids.add(b)
        
    
print(len(trials_uids))

4715


In [ ]:
trials = set([])
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/test/trials'

with open(trials_f, 'r') as f:
    for l in f.readlines():
        a, b,c = l.split()
        a += '-radchn2'
        b += '-radchn2'
        
        trials.append(' '.join([a, b, c]))

### Bandpass

In [5]:
from Process_Data.audio_processing import butter_bandpass_filter
import os
import soundfile as sf
from tqdm import tqdm

```bash
utils/copy_data_dir.sh --utt-suffix -bdpass data/vox1/test data/vox1/test_bandpas
```

In [7]:
subset = 'test_bandpass' #test_radio_chn2
wav_scp = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/wav.scp'%(subset)
new_wavscp = []

lowfreq=300
highfreq=3000

with open(wav_scp, 'r') as f:
    for l in tqdm(f.readlines(), ncols=50):
        uid, upath = l.split()
        y,sr = sf.read(upath)
        
        bp_y = butter_bandpass_filter(
            data=y, cutoff=[lowfreq, highfreq], fs=sr)
        
        # /home/yangwenhao/dataset/voxceleb1/voxceleb1_wav/vox1_test_wav/wav/id10270/5r0dWxy17C8/00001.wav
        bp_upath = upath.replace('/wav/', '/wav_bandpass/')
        bp_upath_dir = os.path.dirname(bp_upath)
        if not os.path.exists(bp_upath_dir):
            os.makedirs(bp_upath_dir)
        
        sf.write(bp_upath, bp_y, samplerate=sr)
        new_wavscp.append(' '.join([uid, bp_upath]))
        # wavscp_dict[uid] = upath
        
with open(wav_scp, 'w') as f:
    for l in new_wavscp:
        f.write(l + '\n')

100%|█████████| 4874/4874 [00:55<00:00, 87.14it/s]


#### Trials_Bandpass

In [8]:
suffix = '' #'-bdpass'
subset = '' #'test_bandpass'
trials_name = '' # 'trials_bandpass'

trials = []
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/trials'%(subset)

with open(trials_f, 'r') as f:
    for l in f.readlines():
        a, b,c = l.split()
        a += suffix
        b += suffix
        
        trials.append(' '.join([a, b, c]))
print(trials[0])

trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/%s'%(subset, trials_name)
with open(trials_f, 'w') as f:
    for l in trials:
        f.write(l + '\n')

id10270-x6uYqmx31kE-00001-bdpass id10270-8jEAjG6SegY-00008-bdpass target


#### Trials_xcross

In [9]:
trials = set([])
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/trials'%(subset)
trials_name = '' # 'trials_bandcross'


with open(trials_f, 'r') as f:
    for l in f.readlines():
        a, b,c = l.split()
        trials.add(' '.join([a+'-bdpass', b, c]))   
        trials.add(' '.join([a, b+'-bdpass', c]))
          
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/%s'%(subset, trials_name)
with open(trials_f, 'w') as f:
    for l in trials:
        f.write(l + '\n')

#### Trials_xy


In [10]:
trials = set([])
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/trials'%(subset)

with open(trials_f, 'r') as f:
    for l in f.readlines():
        a, b,c = l.split()
        trials.add(' '.join([a+'-bdpass', b+'-radchn2', c]))   
        trials.add(' '.join([a+'-radchn2', b+'-bdpass', c]))
          
trials_f = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/trials_bandradio'%(subset)
with open(trials_f, 'w') as f:
    for l in trials:
        f.write(l + '\n')